# ARPO Smoke Test

4 tasks, 2 envs, ~1 hour

**All dependencies explicitly installed**

In [ ]:
# Clone
from getpass import getpass
token = getpass('GitHub token: ')
!git clone https://{token}@github.com/gowathena/arpo_replica.git
%cd arpo_replica
!git checkout arpo-cpu-replicate
!git submodule update --init --recursive

In [ ]:
# Install ALL dependencies explicitly
!pip install -q transformers accelerate
!pip install -q ray omegaconf wandb
!pip install -q tensordict codetiming
!pip install -q mathruler
!pip install -q pylatexenc
!pip install -q qwen-vl-utils pillow

%cd OSWorld
!pip install -q -e .
%cd ..

import sys
sys.path.insert(0, '/content/arpo_replica')

print('✅ All deps installed')

In [ ]:
# Setup Docker for Colab (multiple methods for compatibility)
import subprocess

# Try different Docker startup methods
docker_started = False

# Method 1: service command
try:
    subprocess.run(['sudo', 'service', 'docker', 'start'], check=True, capture_output=True)
    print('✅ Docker started (method 1: service)')
    docker_started = True
except:
    print('Method 1 failed, trying systemctl...')

# Method 2: systemctl command
if not docker_started:
    try:
        subprocess.run(['sudo', 'systemctl', 'start', 'docker'], check=True, capture_output=True)
        print('✅ Docker started (method 2: systemctl)')
        docker_started = True
    except:
        print('Method 2 failed, checking if already running...')

# Verify Docker is working
result = subprocess.run(['docker', 'ps'], capture_output=True, text=True)
if result.returncode == 0:
    print('✅ Docker is running!')
    !docker --version
else:
    print('❌ Docker not available on this Colab runtime')
    print('Try: Runtime → Change runtime type → Make sure GPU is selected')
    raise RuntimeError('Docker not available')

# Pull OSWorld image
!docker pull happysixd/osworld-docker:latest

# Setup Ray
import ray
ray.init(num_cpus=4, num_gpus=1, ignore_reinit_error=True)
print('✅ Ray ready')

# Setup wandb
import wandb, os
from getpass import getpass
os.environ['WANDB_API_KEY'] = getpass('wandb key: ')
wandb.login()
print('✅ wandb ready')

# Update OSWorld for Docker
!sed -i 's/provider_name="vmware"/provider_name="docker"/g' OSWorld/run_uitars.py
!sed -i 's/provider_name="vmware"/provider_name="docker"/g' OSWorld/run_multienv_uitars.py

print('✅ All setup complete!')

In [ ]:
# Config
import yaml

config = {
    'data': {'train_files': 'test_data/osworld_examples/train_smoke_4.json'},
    'algorithm': {'adv_estimator': 'grpo', 'disable_kl': True, 'enable_replay': True},
    'worker': {'actor': {'model': {'model_path': 'ByteDance-Seed/UI-TARS-2B-SFT'}, 'optim': {'lr': 1e-6}}},
    'env': {'num_envs': 2, 'max_steps': 16, 'provider': 'docker'},
    'trainer': {'total_episodes': 1, 'logger': ['console', 'wandb']},
}

with open('smoke.yaml', 'w') as f:
    yaml.dump(config, f)

print('✅ Config ready: 4 tasks, 2 envs, 16 steps')

In [ ]:
# Run
!python -m verl.trainer.main config=smoke.yaml